Questo notebook è tratto dalla documentazione ufficiale di langchain 0.3 e ripercorre quanto già visto nel notebook Ollama_tool_calling_001, utilizzando funzioni di calcolo.<br>

Utilizza il modello open source llama3.2-3B per dimostrare l'utilizzo di tool calling.

Attraverso tool calling un chatmodel può rispondere ad un prompt effettuando una chiamata ad un tool esterno.
Tool calling is a general technique that generates structured output from a model, and you can use it even when you don't intend to invoke any tools. 

I modelli che supportano tool calling sono in grado di generare risposte nel formato previsto da uno schema che viene fornito dall'utente.
A fronte di tale risposta è possibile eseguire un tool esterno (in questo caso una funzione) e poi fornire il risultato di questa esecuzione all'llm per avere la risposta finale


In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2")

Perchè un modello possa invocare dei tools è necessario fornire al modello stesso uno schema che descriva:
- la funzionalità implementata dal tool
- quali sono gli argomenti del tool

I Chat Model che supportano tool calling espongono il metodo bind_tools che consente di passare al modello stesso gli schemas dei tools

Tool schemas possono essere passati come:
- Python functions (with typehints and docstrings),
- Pydantic models
- TypedDict classes
- LangChain Tool objects. 

Vedi https://python.langchain.com/docs/how_to/tool_calling/

Definisco 2 funzioni python decorandole con @tool che consente di convertire una funzione in un tool.

In [2]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

In [3]:
add.args_schema.model_json_schema()

{'description': 'Adds a and b.',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'add',
 'type': 'object'}

I 2 tools vengono passati al llm con la funzione bind_tools().

E' possibile passare diversi argomento a bind_tools per forzare il modello ad utilizzare un determinato tool o per forzarlo ad usare per forza uno dei tools (llm.bind_tools(tools, tool_choice="any"))

In [4]:
tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

Dopo che i tools sono stati associati al modello, verranno passati nel prompt nelle successive invocazioni

In [5]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

# OLtre a HumanMessage sarebbe possibile passare anche SysteMessages
#
messages = [HumanMessage(query)]

# L'invocazione al modello genera gli argomenti che verranno passati successivamente all'invocazione del tool.
ai_msg = llm_with_tools.invoke(messages)
print(ai_msg.tool_calls)

# accodo gli argomenti ai tools ai messaggi
messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '589dbf90-bb4a-4257-9f45-a7531276053b', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'e1e93141-1e56-4577-a371-ebe29681559a', 'type': 'tool_call'}]


Per ciascuno dei tool_call presenti nella risposta, effettuo l'invocazione al tool stesso (utilizzando i parametri presenti).

Anche le risposte all'invocazione dei tools vengono aggiunti ai messaggi che verranno passati al modello per la risposta finale

In [6]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-11T13:12:05.993197111Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8541949880, 'load_duration': 1558660818, 'prompt_eval_count': 231, 'prompt_eval_duration': 3932000000, 'eval_count': 44, 'eval_duration': 3048000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f33bffa0-fac6-48f6-a98a-5a264b1ee55e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '589dbf90-bb4a-4257-9f45-a7531276053b', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'e1e93141-1e56-4577-a371-ebe29681559a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 231, 'output_tokens': 44, 'total_tokens': 275}),
 ToolMessage(content='36', name='multiply', tool_call_id='589dbf90-bb4a-4257-9f45-a7531276053b'),
 To

Invoco infine il modello passando i messaggi con tutti i dati necessari per produrre la risposta finale

In [7]:
response = llm_with_tools.invoke(messages)
response

AIMessage(content='The result of 3 * 12 is 36.\nThe result of 11 + 49 is 60.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-11T13:12:42.491280526Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3489496252, 'load_duration': 16296497, 'prompt_eval_count': 125, 'prompt_eval_duration': 1226000000, 'eval_count': 25, 'eval_duration': 1682000000, 'message': Message(role='assistant', content='The result of 3 * 12 is 36.\nThe result of 11 + 49 is 60.', images=None, tool_calls=None)}, id='run-d29fb5b6-79a5-4109-9b2e-c8610bfe4d52-0', usage_metadata={'input_tokens': 125, 'output_tokens': 25, 'total_tokens': 150})

In [8]:
print(response.content)

The result of 3 * 12 is 36.
The result of 11 + 49 is 60.


In caso di calcoli più difficili è possibile fornire esempi al modello.

Vedi:
https://python.langchain.com/docs/how_to/tools_few_shot/

VERIFICARE PERCHE' NON FUNZIONA (for

In [9]:
llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"
).tool_calls

[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': '9625a5c2-7cad-481a-ad54-d01ec52d13ea',
  'type': 'tool_call'},
 {'name': 'subtract',
  'args': {'a': 'result', 'b': 20},
  'id': '684cf20a-fa3c-4559-b375-f50d845297a1',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 7, 'b': 4},
  'id': 'cfd533a6-df03-4846-9478-92db010ea924',
  'type': 'tool_call'}]

In [10]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "Multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "Add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 minus 20").tool_calls

[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': '74a9d413-7d06-4f12-bf16-cba556dfdd68',
  'type': 'tool_call'},
 {'name': 'subtract',
  'args': {'x': 165952, 'y': 20},
  'id': '621e1d1d-ecbe-4993-8f9e-99c5424c126d',
  'type': 'tool_call'}]